# 패키지들은 연습 문제를 풀며 연습하기 위해 별도 로드 X

# Question 1.
- 서비스 이탈예측 데이터.
- 2 유형 환경과 일치시키기 위해 y_train과 x_train을 하나의 train 데이터 셋으로 병합하여 진행.

## 데이터 병합 및 확인 단계

In [43]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/churnk/X_test.csv')

train = pd.concat([x_train, y_train['Exited']], axis = 1)
display(train.info())
display(x_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6499 entries, 0 to 6498
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       6499 non-null   int64  
 1   Surname          6499 non-null   object 
 2   CreditScore      6499 non-null   int64  
 3   Geography        6499 non-null   object 
 4   Gender           6499 non-null   object 
 5   Age              6499 non-null   int64  
 6   Tenure           6499 non-null   int64  
 7   Balance          6499 non-null   float64
 8   NumOfProducts    6499 non-null   int64  
 9   HasCrCard        6499 non-null   int64  
 10  IsActiveMember   6499 non-null   int64  
 11  EstimatedSalary  6499 non-null   float64
 12  Exited           6499 non-null   int64  
dtypes: float64(2), int64(8), object(3)
memory usage: 660.2+ KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3501 entries, 0 to 3500
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   CustomerId       3501 non-null   int64  
 1   Surname          3501 non-null   object 
 2   CreditScore      3501 non-null   int64  
 3   Geography        3501 non-null   object 
 4   Gender           3501 non-null   object 
 5   Age              3501 non-null   int64  
 6   Tenure           3501 non-null   int64  
 7   Balance          3501 non-null   float64
 8   NumOfProducts    3501 non-null   int64  
 9   HasCrCard        3501 non-null   int64  
 10  IsActiveMember   3501 non-null   int64  
 11  EstimatedSalary  3501 non-null   float64
dtypes: float64(2), int64(7), object(3)
memory usage: 328.3+ KB


None

In [44]:
# 1. 데이터의 통계량 확인
display(train.describe(include = ['object', 'category']))
display(train.describe())

# 결과 변수
#   - count: 결측치를 제외한 전체 데이터의 수 (행의 수와 일치).
#   - unique: 서로 다른 범주의 수 (범주 종류의 수).
#   - top: 최빈 범주
#   - freq: 최빈 범주의 개수.

,Surname,Geography,Gender
count,6499,6499,6499
unique,2289,3,4
top,Brown,France,Male
freq,21,3227,3485


,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,6.499000e+03,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000,6499.000000
mean,1.569157e+07,650.396830,38.957070,5.041545,76836.581068,1.519772,0.708878,0.514387,100346.564524,0.203724
std,7.187584e+04,96.618957,10.502803,2.891779,62407.570894,0.578975,0.454314,0.499831,57944.655305,0.402797
min,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.000000,0.000000,11.580000,0.000000
25%,1.562949e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.000000,0.000000,50907.565000,0.000000
50%,1.569181e+07,651.000000,37.000000,5.000000,97560.160000,1.000000,1.000000,1.000000,100496.840000,0.000000
75%,1.575358e+07,718.000000,44.000000,8.000000,127844.690000,2.000000,1.000000,1.000000,150480.155000,0.000000
max,1.581566e+07,850.000000,92.000000,10.000000,238387.560000,4.000000,1.000000,1.000000,199970.740000,1.000000


## 변수 선택 단계

In [45]:
# Surname과 CustomerId는 제외
#   - Surname: 범주의 종류가 2289개로 의미없는 정보임.
#   - CustomerId: Id 컬럼이기에 제외
train_df = train.drop(columns = ['Surname', 'CustomerId'])
x_test_df = x_test.drop(columns = ['Surname', 'CustomerId'])

## 설명변수와 결과변수 분리

In [46]:
# train_df의 설명변수와 결과변수를 분리
X = train_df.drop(columns = 'Exited')
y = train_df['Exited']

## Encoding

In [52]:
# 범주형 변수만 따로 추출
target = train_df.drop(columns = 'Exited')
cat_lst = target.select_dtypes(exclude = 'number').columns.tolist()
cat_lst

# 원-핫 인코딩 적용
#   - drop_first = False
#       - False: 기준 범주 없이 모든 범주를 0과 1로 encoding 할 때 사용한다. 이는 기준 범주 없이 개별 범주로 처리하기 때문.
#       - True: 통계모델에서 기준 범주를 설정할 때 사용한다. 통계에서는 기준 범주를 기준으로하여 변화량을 측정하기 때문. (이 경우 기준범주가 0이 된다).
train_encoded = pd.get_dummies(X, columns = cat_lst, drop_first = False)
x_test_encoded = pd.get_dummies(x_test_df, columns = cat_lst, drop_first = False)

## 데이터를 분리하고 학습결과 확인.

In [53]:
from sklearn.model_selection import train_test_split

# train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_encoded, y, test_size = 0.3, random_state = 42, stratify = y)

## 학습

In [54]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 결과 지표들 확인

In [55]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1. 예측
val_pred_label = rf.predict(X_val)               # 클래스 예측
val_pred_proba = rf.predict_proba(X_val)[:, 1]  # 양성 클래스 확률 (AUC에 사용)

# 2. 각 지표 출력
print("📊 모델 성능 평가 결과:")
print(f"Accuracy : {accuracy_score(y_val, val_pred_label):.4f}")
print(f"Precision: {precision_score(y_val, val_pred_label):.4f}")
print(f"Recall   : {recall_score(y_val, val_pred_label):.4f}")
print(f"F1-score : {f1_score(y_val, val_pred_label):.4f}")
print(f"AUC-ROC  : {roc_auc_score(y_val, val_pred_proba):.4f}")

# 3. 추가 보고서
print("\n📋 Classification Report:")
print(classification_report(y_val, val_pred_label))

# 4. 혼동행렬
print("🧮 Confusion Matrix:")
print(confusion_matrix(y_val, val_pred_label))


📊 모델 성능 평가 결과:
Accuracy : 0.8605
Precision: 0.7962
Recall   : 0.4232
F1-score : 0.5526
AUC-ROC  : 0.8467

📋 Classification Report:
              precision    recall  f1-score   support

           0       0.87      0.97      0.92      1553
           1       0.80      0.42      0.55       397

    accuracy                           0.86      1950
   macro avg       0.83      0.70      0.74      1950
weighted avg       0.85      0.86      0.84      1950

🧮 Confusion Matrix:
[[1510   43]
 [ 229  168]]


## test 데이터에 적용하여 csv파일 생성.

In [57]:
test_pred_label = rf.predict(x_test_encoded)
test_pred_proba = rf.predict_proba(x_test_encoded)[ : , 1]

# ID 생성
pd.DataFrame({'pred' : test_pred_label}).to_csv('label.csv', index=True, index_label = 'ID')
pd.DataFrame({'pred' : test_pred_proba}).to_csv('proba.csv', index=True, index_label = 'ID')

---

# Question 2.
- 이직여부 판단 데이터
- 병합된 데이터가 들어옴을 가정.

## 데이터 병합 및 확인

In [67]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/HRdata/X_test.csv')

train_total = pd.concat([x_train, y_train['target']], axis = 1)
display(train_total.head())
display(x_test.head())

,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours,target
0,25298,city_138,0.836,Male,No relevent experience,Full time course,High School,NaN,5,100-500,Pvt Ltd,1,45,0.0
1,4241,city_160,0.920,Male,No relevent experience,Full time course,High School,NaN,5,NaN,NaN,1,17,1.0
2,24086,city_57,0.866,Male,No relevent experience,no_enrollment,Graduate,STEM,10,NaN,NaN,1,50,0.0
3,26773,city_16,0.910,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,>4,135,0.0
4,32325,city_143,0.740,NaN,No relevent experience,Full time course,Graduate,STEM,5,NaN,NaN,never,17,1.0


,enrollee_id,city,city_development_index,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job,training_hours
0,7129,city_23,0.899,Male,No relevent experience,no_enrollment,Graduate,STEM,2,NaN,NaN,1,23
1,31037,city_44,0.725,Male,No relevent experience,Part time course,Masters,STEM,5,NaN,NaN,never,39
2,22179,city_103,0.920,Male,No relevent experience,Full time course,Graduate,STEM,7,NaN,NaN,2,262
3,29724,city_50,0.896,Male,No relevent experience,Full time course,Graduate,STEM,7,NaN,NaN,never,78
4,17977,city_149,0.689,Female,Has relevent experience,Full time course,Masters,STEM,19,50-99,Pvt Ltd,1,125


# 데이터의 정보확인

In [68]:
display(train_total.info())
display(x_test.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             12452 non-null  int64  
 1   city                    12452 non-null  object 
 2   city_development_index  12452 non-null  float64
 3   gender                  9535 non-null   object 
 4   relevent_experience     12452 non-null  object 
 5   enrolled_university     12195 non-null  object 
 6   education_level         12137 non-null  object 
 7   major_discipline        10586 non-null  object 
 8   experience              12415 non-null  object 
 9   company_size            8600 non-null   object 
 10  company_type            8471 non-null   object 
 11  last_new_job            12179 non-null  object 
 12  training_hours          12452 non-null  int64  
 13  target                  12452 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6706 entries, 0 to 6705
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   enrollee_id             6706 non-null   int64  
 1   city                    6706 non-null   object 
 2   city_development_index  6706 non-null   float64
 3   gender                  5115 non-null   object 
 4   relevent_experience     6706 non-null   object 
 5   enrolled_university     6577 non-null   object 
 6   education_level         6561 non-null   object 
 7   major_discipline        5759 non-null   object 
 8   experience              6678 non-null   object 
 9   company_size            4620 non-null   object 
 10  company_type            4547 non-null   object 
 11  last_new_job            6556 non-null   object 
 12  training_hours          6706 non-null   int64  
dtypes: float64(1), int64(2), object(10)
memory usage: 681.2+ KB


None

## 통계량 확인

In [69]:
display(train_total.describe())
display(train_total.describe(include = ['object', 'category']))

,enrollee_id,city_development_index,training_hours,target
count,12452.000000,12452.000000,12452.000000,12452.000000
mean,16966.301076,0.829709,65.604481,0.249358
std,9646.223269,0.123087,60.430340,0.432659
min,1.000000,0.448000,1.000000,0.000000
25%,8591.500000,0.740000,23.000000,0.000000
50%,17177.500000,0.910000,47.000000,0.000000
75%,25327.750000,0.920000,88.000000,0.000000
max,33380.000000,0.949000,336.000000,1.000000


,city,gender,relevent_experience,enrolled_university,education_level,major_discipline,experience,company_size,company_type,last_new_job
count,12452,9535,12452,12195,12137,10586,12415,8600,8471,12179
unique,122,3,2,3,5,6,22,8,6,6
top,city_103,Male,Has relevent experience,no_enrollment,Graduate,STEM,>20,50-99,Pvt Ltd,1
freq,2839,8595,8953,8975,7540,9414,2179,2000,6413,5157


## 결측치 처리.
- 현재 object형 컬러만 결측치가 존재.

In [86]:
# object이거나 category형인 컬럼만 필터링.
cat_lst = train_total.select_dtypes(include = ['object', 'category']).columns
num_lst = train_total.select_dtypes(include = 'number').columns
train_cat = train_total.loc[ : , cat_lst]
train_num = train_total.loc[ : , num_lst]

# fillna(value, inplace = )를 통해 최빈값으로 대체
#   - x.mode(dropna = True)[0]: 최빈값 중 첫 번째 값을 반환해서 결측치 대신 채움
#       - dropna: True는 결측치를 제외한 최빈값을 반환.
#   - inplace: True인 경우 원본 데이터프레임을 변환함. False인 경우 새로운 데이터프레임을 반환함.
train_cat_nn = train_total[cat_lst].apply(lambda x: x.fillna(x.mode(dropna = True)[0], inplace = False))

# 결측값 처리 후 다시 수치형 변수들과 병합.
train_nn = pd.concat([train_cat_nn, train_num], axis = 1)

# 결과 확인
display(train_nn.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12452 entries, 0 to 12451
Data columns (total 14 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   city                    12452 non-null  object 
 1   gender                  12452 non-null  object 
 2   relevent_experience     12452 non-null  object 
 3   enrolled_university     12452 non-null  object 
 4   education_level         12452 non-null  object 
 5   major_discipline        12452 non-null  object 
 6   experience              12452 non-null  object 
 7   company_size            12452 non-null  object 
 8   company_type            12452 non-null  object 
 9   last_new_job            12452 non-null  object 
 10  enrollee_id             12452 non-null  int64  
 11  city_development_index  12452 non-null  float64
 12  training_hours          12452 non-null  int64  
 13  target                  12452 non-null  float64
dtypes: float64(2), int64(2), object(10)
me

None

## Encoding 적용

In [101]:
# 결과변수 분리
X = train_nn.drop(columns = 'target')
y = train_nn['target']

# get_dummies를 통한 인코딩
train_encoded = pd.get_dummies(X, columns = cat_lst, drop_first = False)
x_test_encoded = pd.get_dummies(x_test, columns = cat_lst, drop_first = False)

# Encoding 된 범주의 수가 일치하지 않는 경우 맞춰준다.
train_encoded, x_test_encoded = train_encoded.align(x_test_encoded, join='left', axis=1, fill_value=0)

## 데이터 분리

In [96]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(train_encoded, y, test_size = 0.3, random_state = 42)

## 학습

In [97]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 결과확인

In [98]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 1. 예측
val_pred_label = rf.predict(X_val)               # 클래스 예측
val_pred_proba = rf.predict_proba(X_val)[:, 1]  # 양성 클래스 확률 (AUC에 사용)

# 2. 각 지표 출력
print("📊 모델 성능 평가 결과:")
print(f"Accuracy : {accuracy_score(y_val, val_pred_label):.4f}")
print(f"Precision: {precision_score(y_val, val_pred_label):.4f}")
print(f"Recall   : {recall_score(y_val, val_pred_label):.4f}")
print(f"F1-score : {f1_score(y_val, val_pred_label):.4f}")
print(f"AUC-ROC  : {roc_auc_score(y_val, val_pred_proba):.4f}")

# 3. 추가 보고서
print("\n📋 Classification Report:")
print(classification_report(y_val, val_pred_label))

# 4. 혼동행렬
print("🧮 Confusion Matrix:")
print(confusion_matrix(y_val, val_pred_label))

📊 모델 성능 평가 결과:
Accuracy : 0.7642
Precision: 0.5419
Recall   : 0.3733
F1-score : 0.4421
AUC-ROC  : 0.7530

📋 Classification Report:
              precision    recall  f1-score   support

         0.0       0.81      0.89      0.85      2801
         1.0       0.54      0.37      0.44       935

    accuracy                           0.76      3736
   macro avg       0.68      0.63      0.65      3736
weighted avg       0.74      0.76      0.75      3736

🧮 Confusion Matrix:
[[2506  295]
 [ 586  349]]


## CSV 파일 생성

In [100]:
test_pred_label = rf.predict(x_test_encoded)
test_pred_proba = rf.predict_proba(x_test_encoded)[ : , 1]

pd.DataFrame({'pred' : test_pred_label}).to_csv('label_csv', index = True, index_label = 'ID')
pd.DataFrame({'pred_proba' : test_pred_proba}).to_csv('proba_csv', index = True, index_label = 'ID')

---

# Question 3.
- 정시 배송 여부 판단
- 지금부터는 병합하지 않고 풀이

## 데이터 로드

In [18]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv')

#y_train에 ID가 포함돼 있어서 제거.
y_train = y_train.drop(columns = 'ID')

## 데이터 통계량 확인

In [19]:
display(x_train.describe())
display(x_train.describe(include = 'object'))

,ID,Customer_rating,Cost_of_the_Product,Prior_purchases,Discount_offered,Weight_in_gms
count,6598.000000,6598.000000,6598.000000,6598.000000,6598.000000,6598.000000
mean,5476.977266,2.991361,210.393149,3.577751,13.353592,3604.191119
std,3172.946154,1.409624,48.258089,1.511394,16.187267,1635.697627
min,1.000000,1.000000,96.000000,2.000000,1.000000,1001.000000
25%,2731.250000,2.000000,170.000000,3.000000,4.000000,1834.250000
50%,5476.000000,3.000000,214.000000,3.000000,7.000000,4119.500000
75%,8187.750000,4.000000,251.000000,4.000000,10.000000,5027.500000
max,10998.000000,5.000000,310.000000,10.000000,65.000000,7684.000000


,Warehouse_block,Mode_of_Shipment,Customer_care_calls,Product_importance,Gender
count,6598,6598,6598,6598,6598
unique,5,3,6,3,2
top,F,Ship,4,low,F
freq,2262,4512,2115,3162,3311


## 결측치와 정보 확인

In [20]:
display(x_train.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6598 entries, 0 to 6597
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   ID                   6598 non-null   int64 
 1   Warehouse_block      6598 non-null   object
 2   Mode_of_Shipment     6598 non-null   object
 3   Customer_care_calls  6598 non-null   object
 4   Customer_rating      6598 non-null   int64 
 5   Cost_of_the_Product  6598 non-null   int64 
 6   Prior_purchases      6598 non-null   int64 
 7   Product_importance   6598 non-null   object
 8   Gender               6598 non-null   object
 9   Discount_offered     6598 non-null   int64 
 10  Weight_in_gms        6598 non-null   int64 
dtypes: int64(6), object(5)
memory usage: 567.1+ KB


None

## Encoding

In [21]:
# 범주형 변수명만 추출
cat_lst = x_train.select_dtypes(include = ['object', 'category']).columns.tolist()
cat_lst

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(
    handle_unknown = 'ignore',
    sparse_output = False
)

# Encoding 전에 수치형 변수일단 필터링 해두기.
train_num = x_train.select_dtypes(include = 'number')
test_num = x_test.select_dtypes(include = 'number')

# OHE를 통해 train_encoded 생성.
train_encoded = ohe.fit_transform(x_train[cat_lst])

# ohe로 반환된 train_encoded는 numpy.ndarray이기에 데이터프레임으로 변환
train_cat = pd.DataFrame(
    train_encoded, # Encoding된 numpy.ndarray형의 데이터.
    columns = ohe.get_feature_names_out(cat_lst), # 컬럼이름 복원.
    index = x_train.index # 원래의 인덱스를 그대로사용.
)

# x_test Encoding
test_encoded = ohe.transform(x_test[cat_lst])

# Encoding된 test_encoded를 데이터프레임으로 변환
test_cat = pd.DataFrame(
    test_encoded,
    columns = ohe.get_feature_names_out(cat_lst),
    index = x_test.index
)

# 데이터 다시 통합
X = pd.concat([train_cat, train_num], axis = 1)
y = y_train
test_final = pd.concat([test_cat, test_num], axis = 1)

## 데이터를 분할

In [22]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.3, random_state = 42)

## 예측 수행

In [23]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

c:\Users\ashle\OneDrive\바탕 화면\Certifications\빅분기\venv\Lib\site-packages\sklearn\base.py:1363: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)


,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 결과 확인

In [30]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 예측 수행
val_pred_label = rf.predict(X_val)
val_pred_proba = rf.predict_proba(X_val)[ : , 1]

# 예측 결과 출력
print("모델 성능 평가 확인")
print(f"Accuracy   : {accuracy_score(y_val, val_pred_label):.4f}")
print(f"Precision  : {precision_score(y_val, val_pred_label):.4f}")
print(f"Recall     : {recall_score(y_val, val_pred_label):.4f}")
print(f"F1-score   : {f1_score(y_val, val_pred_label):.4f}")
print(f"AUC-ROC    : {roc_auc_score(y_val, val_pred_proba):.4f}")

# 추가 보고서
print("\n Classification Report: ")
print(classification_report(y_val, val_pred_label))

# 혼동행렬
print('\n Confusion matrix: ')
print(confusion_matrix(y_val, val_pred_label))

모델 성능 평가 확인
Accuracy   : 0.6525
Precision  : 0.7682
Recall     : 0.5928
F1-score   : 0.6692
AUC-ROC    : 0.7271

 Classification Report: 
              precision    recall  f1-score   support

           0       0.55      0.74      0.63       806
           1       0.77      0.59      0.67      1174

    accuracy                           0.65      1980
   macro avg       0.66      0.67      0.65      1980
weighted avg       0.68      0.65      0.65      1980


 Confusion matrix: 
[[596 210]
 [478 696]]


## CVS 파일 생성

In [35]:
test_pred_label = rf.predict(test_final)
test_pred_proba = rf.predict_proba(test_final)

pd.DataFrame({"pred" : test_pred_label}).to_csv("label_csv", index = True, index_label = 'ID')
pd.DataFrame({"pred" : test_pred_label}).to_csv("proba_csv", index = True, index_label = 'ID')

---

# Question 4.
- 성인 건강검진 예측
- 데이터 병합하지 않고 진행.

## 데이터 로드 및 확인

In [53]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/smoke/x_test.csv')

y_train = y_train['흡연상태']

display(x_train.info())
display(y_train.info())
display(x_test.info())
print("==========================================================================================")
print("==========================================================================================")
display(x_train.isna().sum())
print("==========================================================================================")
print("==========================================================================================")
display(x_train.describe())
display(x_train.describe(include = ['object', 'category']))

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44553 entries, 0 to 44552
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           44553 non-null  int64  
 1   성별코드         44553 non-null  object 
 2   연령대코드(5세단위)  44553 non-null  int64  
 3   신장(5Cm단위)    44553 non-null  int64  
 4   체중(5Kg단위)    44553 non-null  int64  
 5   허리둘레         44553 non-null  float64
 6   시력(좌)        44553 non-null  float64
 7   시력(우)        44553 non-null  float64
 8   청력(좌)        44553 non-null  float64
 9   청력(우)        44553 non-null  float64
 10  수축기혈압        44553 non-null  float64
 11  이완기혈압        44553 non-null  float64
 12  식전혈당(공복혈당)   44553 non-null  float64
 13  총콜레스테롤       44553 non-null  float64
 14  트리글리세라이드     44553 non-null  float64
 15  HDL콜레스테롤     44553 non-null  float64
 16  LDL콜레스테롤     44553 non-null  float64
 17  혈색소          44553 non-null  float64
 18  요단백          44553 non-null  float64
 19  혈청크레

None

<class 'pandas.core.series.Series'>
RangeIndex: 44553 entries, 0 to 44552
Series name: 흡연상태
Non-Null Count  Dtype
--------------  -----
44553 non-null  int64
dtypes: int64(1)
memory usage: 348.2 KB


None

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11139 entries, 0 to 11138
Data columns (total 26 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ID           11139 non-null  int64  
 1   성별코드         11139 non-null  object 
 2   연령대코드(5세단위)  11139 non-null  int64  
 3   신장(5Cm단위)    11139 non-null  int64  
 4   체중(5Kg단위)    11139 non-null  int64  
 5   허리둘레         11139 non-null  float64
 6   시력(좌)        11139 non-null  float64
 7   시력(우)        11139 non-null  float64
 8   청력(좌)        11139 non-null  float64
 9   청력(우)        11139 non-null  float64
 10  수축기혈압        11139 non-null  float64
 11  이완기혈압        11139 non-null  float64
 12  식전혈당(공복혈당)   11139 non-null  float64
 13  총콜레스테롤       11139 non-null  float64
 14  트리글리세라이드     11139 non-null  float64
 15  HDL콜레스테롤     11139 non-null  float64
 16  LDL콜레스테롤     11139 non-null  float64
 17  혈색소          11139 non-null  float64
 18  요단백          11139 non-null  float64
 19  혈청크레

None

ID             0
성별코드           0
연령대코드(5세단위)    0
신장(5Cm단위)      0
체중(5Kg단위)      0
허리둘레           0
시력(좌)          0
시력(우)          0
청력(좌)          0
청력(우)          0
수축기혈압          0
이완기혈압          0
식전혈당(공복혈당)     0
총콜레스테롤         0
트리글리세라이드       0
HDL콜레스테롤       0
LDL콜레스테롤       0
혈색소            0
요단백            0
혈청크레아티닌        0
(혈청지오티)AST     0
(혈청지오티)ALT     0
감마지티피          0
구강검진수검여부       0
치아우식증유무        0
치석             0
dtype: int64

,ID,연령대코드(5세단위),신장(5Cm단위),체중(5Kg단위),허리둘레,시력(좌),시력(우),청력(좌),청력(우),수축기혈압,...,트리글리세라이드,HDL콜레스테롤,LDL콜레스테롤,혈색소,요단백,혈청크레아티닌,(혈청지오티)AST,(혈청지오티)ALT,감마지티피,치아우식증유무
count,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000,44553.00000,44553.000000,44553.000000,...,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000,44553.000000
mean,27833.607389,44.210603,164.657038,65.883442,82.077247,1.011734,1.008134,1.02561,1.026036,121.529055,...,126.721770,57.288421,115.037663,14.622194,1.087020,0.886097,26.213678,27.084731,40.065517,0.213521
std,16064.577297,12.089077,9.198571,12.823676,9.278288,0.488131,0.488762,0.15797,0.159245,13.688748,...,71.611991,14.795235,40.937878,1.564872,0.404944,0.226092,19.087106,31.754805,50.723521,0.409797
min,0.000000,20.000000,130.000000,30.000000,51.000000,0.100000,0.100000,1.00000,1.000000,71.000000,...,8.000000,4.000000,1.000000,4.900000,1.000000,0.100000,6.000000,1.000000,1.000000,0.000000
25%,13938.000000,40.000000,160.000000,55.000000,76.000000,0.800000,0.800000,1.00000,1.000000,112.000000,...,74.000000,47.000000,92.000000,13.600000,1.000000,0.800000,19.000000,15.000000,17.000000,0.000000
50%,27819.000000,40.000000,165.000000,65.000000,82.000000,1.000000,1.000000,1.00000,1.000000,120.000000,...,108.000000,55.000000,113.000000,14.800000,1.000000,0.900000,23.000000,21.000000,26.000000,0.000000
75%,41738.000000,55.000000,170.000000,75.000000,88.000000,1.200000,1.200000,1.00000,1.000000,130.000000,...,160.000000,66.000000,136.000000,15.700000,1.000000,1.000000,29.000000,31.000000,44.000000,0.000000
max,55690.000000,85.000000,190.000000,135.000000,129.000000,9.900000,9.900000,2.00000,2.000000,240.000000,...,999.000000,618.000000,1860.000000,21.100000,6.000000,11.600000,1311.000000,2914.000000,999.000000,1.000000


,성별코드,구강검진수검여부,치석
count,44553,44553,44553
unique,2,1,2
top,M,Y,Y
freq,28349,44553,24762


## Encoding

In [54]:
from sklearn.preprocessing import OneHotEncoder

# OneHotEncoder 생성
ohe = OneHotEncoder(
    handle_unknown = 'ignore',
    sparse_output = False
)

# 수치형 데이터와 변수형 데이터 분리
# 범주형 변수만 필터링
cat_lst = x_train.select_dtypes(include = ['object', 'category']).columns.tolist()

train_num = x_train.drop(columns = cat_lst)
test_num = x_test.drop(columns = cat_lst)

# 범주형 변수 encoding
train_encoded = ohe.fit_transform(x_train[cat_lst])
train_cat = pd.DataFrame(
    train_encoded,
    columns = ohe.get_feature_names_out(cat_lst),
    index = x_train.index
)

test_encoded = ohe.transform(x_test[cat_lst])
test_cat = pd.DataFrame(
    test_encoded,
    columns = ohe.get_feature_names_out(cat_lst),
    index = x_test.index
)

# 하나의 데이터 프레임으로 병합
X = pd.concat([train_num, train_cat], axis = 1)
test_final = pd.concat([test_num, test_cat], axis = 1)


## 데이터 분할

In [55]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size = 0.3, random_state = 42)

## 모델 생성 후 학습

In [56]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 예측 및 평가지표 출력

In [60]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix, roc_auc_score

# 예측수행
val_pred_label = rf.predict(X_val)
val_pred_proba = rf.predict_proba(X_val)[ : , 1]

# 평가지표출력
print(f"Accuracy    : {accuracy_score(y_val, val_pred_label)}:.4f")
print(f"Precision   : {precision_score(y_val, val_pred_label)}:.4f")
print(f"Recall      : {recall_score(y_val, val_pred_label)}:.4f")
print(f"F1_Score    : {f1_score(y_val, val_pred_label)}:.4f")
print(f"ROC-AUC     : {roc_auc_score(y_val, val_pred_proba)}:.4f")

# Classification report
print("\n Classification Report: ")
print(classification_report(y_val, val_pred_label))

# Confusion Matrix
print("\n Confusion Matrix: ")
print(confusion_matrix(y_val, val_pred_label))

Accuracy    : 0.7581176118509652:.4f
Precision   : 0.6603593272171254:.4f
Recall      : 0.7035227041335776:.4f
F1_Score    : 0.681258010450557:.4f
ROC-AUC     : 0.8379321647381344:.4f

 Classification Report: 
              precision    recall  f1-score   support

           0       0.82      0.79      0.81      8455
           1       0.66      0.70      0.68      4911

    accuracy                           0.76     13366
   macro avg       0.74      0.75      0.74     13366
weighted avg       0.76      0.76      0.76     13366


 Confusion Matrix: 
[[6678 1777]
 [1456 3455]]


## CSV 파일 생성

In [62]:
test_pred_label = rf.predict(test_final)
test_pred_proba = rf.predict_proba(test_final)[ : , 1]

pd.DataFrame({'pred' : test_pred_label}).to_csv("label_csv", index = True, index_label = 'ID')
pd.DataFrame({'pred' : test_pred_proba}).to_csv("proba_csv", index = True, index_label = 'ID')

---

# Question 5.
- 자동차 보험가입 예측 데이터
- 데이터는 분할된 채 그대로 풀이

In [64]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/insurance/x_test.csv')

# 결과변수만 필터링
y_train = y_train['Response']

# 결측치 확인
display(x_train.isna().sum())

# 통계량 확인
print("======================================================================================")
print("======================================================================================")
display(x_train.describe(include = 'object'))
display(x_train.describe())


ID                           0
Gender                       0
Age                          0
Driving_License              0
Region_Code                  0
Previously_Insured           0
Vehicle_Age                  0
Vehicle_Damage               0
Annual_Premium               0
Policy_Sales_Channel         0
Vintage                      0
id                      304887
dtype: int64

,Gender,Vehicle_Age,Vehicle_Damage
count,304887,304887,304887
unique,2,3,2
top,Male,1-2 Year,Yes
freq,164764,160409,153869


,ID,Age,Driving_License,Region_Code,Previously_Insured,Annual_Premium,Policy_Sales_Channel,Vintage,id
count,304887.000000,304887.000000,304887.000000,304887.000000,304887.000000,304887.000000,304887.000000,304887.000000,0.0
mean,190511.074044,38.828304,0.997898,26.382650,0.458258,30576.579598,112.044954,154.319466,NaN
std,110035.080783,15.504898,0.045804,13.236626,0.498255,17265.678980,54.198329,83.625500,NaN
min,0.000000,20.000000,0.000000,0.000000,0.000000,2630.000000,1.000000,10.000000,NaN
25%,95216.000000,25.000000,1.000000,15.000000,0.000000,24414.500000,29.000000,82.000000,NaN
50%,190452.000000,36.000000,1.000000,28.000000,0.000000,31663.000000,133.000000,154.000000,NaN
75%,285790.500000,49.000000,1.000000,35.000000,1.000000,39410.000000,152.000000,227.000000,NaN
max,381107.000000,85.000000,1.000000,52.000000,1.000000,540165.000000,163.000000,299.000000,NaN


## 결측치 처리

In [65]:
# id 컬럼의 거의 모든 데이터가 NaN이기에 완전히 제거.
x_train = x_train.drop(columns = 'id')
x_test = x_test.drop(columns = 'id')

## Encoding

In [68]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(
    handle_unknown = 'ignore',
    sparse_output = False
)

# 범주형 변수만 추출
cat_lst = x_train.select_dtypes(include = ['object', 'category']).columns.tolist()

# 수치형 데이터 별도로 관리
train_num = x_train.drop(columns = cat_lst)
test_num = x_test.drop(columns = cat_lst)

# train 데이터 encoding
train_encoded = ohe.fit_transform(x_train[cat_lst])
train_cat = pd.DataFrame(
    train_encoded,
    columns = ohe.get_feature_names_out(cat_lst),
    index = x_train.index
)

test_encoded = ohe.transform(x_test[cat_lst])
test_cat = pd.DataFrame(
    test_encoded,
    columns = ohe.get_feature_names_out(cat_lst),
    index = x_test.index
)

# 하나의 데이터프레임으로 생성
X = pd.concat([train_num, train_cat], axis = 1)
test_final = pd.concat([test_num, test_cat], axis = 1)

## 데이터 분할

In [ ]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size = 0.3, random_state = 42)

## 모델 생성 및 적용

In [70]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 평가 지표 출력

In [76]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

# 예측 결과 반환
val_pred_label = rf.predict(X_val)
val_pred_proba = rf.predict_proba(X_val)[ : , 1]

# 결과지표 출력
print(f"Accuracy    : {accuracy_score(y_val, val_pred_label):.4f}")
print(f"Precision   : {precision_score(y_val, val_pred_label):.4f}")
print(f"Recall      : {recall_score(y_val, val_pred_label):.4f}")
print(f"F1-score    : {f1_score(y_val, val_pred_label):.4f}")
print(f"ROC_AUC     : {roc_auc_score(y_val, val_pred_proba):.4f}")

# Classification report
print('\n Classification report')
print(classification_report(y_val, val_pred_label))

# Confusion matrix
print('\n Confusion Matrix')
print(confusion_matrix(y_val, val_pred_label))


Accuracy    : 0.8722
Precision   : 0.3823
Recall      : 0.0759
F1-score    : 0.1267
ROC_AUC     : 0.8412

 Classification report
              precision    recall  f1-score   support

           0       0.88      0.98      0.93     80298
           1       0.38      0.08      0.13     11169

    accuracy                           0.87     91467
   macro avg       0.63      0.53      0.53     91467
weighted avg       0.82      0.87      0.83     91467


 Confusion Matrix
[[78928  1370]
 [10321   848]]


## CSV 파일 출력

In [77]:
test_pred_label = rf.predict(test_final)
test_pred_proba = rf.predict_proba(test_final)[ : , 1]

pd.DataFrame({'pred' : test_pred_label}).to_csv('label_scv', index = True, index_label = 'ID')
pd.DataFrame({'pred' : test_pred_proba}).to_csv('proba_csv', index = True, index_label = 'ID')

---

# Question 6.
- 비행탑승 경험 만족도 분류
- 데이터 병합 없이 진행

In [2]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/airline/x_test.csv')

# 결과변수 encoding 필요.
y_train = y_train['satisfaction']

# id 컬럼 제거 필요.
# Arrival Delay in Minutes 컬럼 결측치 존재.
# 설명변수와 결과변수 모두 `    Encoding 필요.
display(x_train.isna().sum())
display(x_train.describe(include = ['object', 'category']))
display(x_train.describe())

# 제거 됐는지 확인
x_train = x_train.drop(columns = 'id')
x_test = x_test.drop(columns = 'id')
if 'id' in x_train.columns.tolist():
    print(True)
else:
    print(False)

ID                                       0
Gender                                   0
Customer Type                            0
Age                                      0
Type of Travel                           0
Class                                    0
Flight Distance                          0
Inflight wifi service                    0
Departure/Arrival time convenient        0
Ease of Online booking                   0
Gate location                            0
Food and drink                           0
Online boarding                          0
Seat comfort                             0
Inflight entertainment                   0
On-board service                         0
Leg room service                         0
Baggage handling                         0
Checkin service                          0
Inflight service                         0
Cleanliness                              0
Departure Delay in Minutes               0
Arrival Delay in Minutes               256
id         

,Gender,Customer Type,Type of Travel,Class
count,83123,83123,83123,83123
unique,2,2,2,3
top,Female,Loyal Customer,Business travel,Business
freq,42284,67951,57370,39791


,ID,Age,Flight Distance,Inflight wifi service,Departure/Arrival time convenient,Ease of Online booking,Gate location,Food and drink,Online boarding,Seat comfort,Inflight entertainment,On-board service,Leg room service,Baggage handling,Checkin service,Inflight service,Cleanliness,Departure Delay in Minutes,Arrival Delay in Minutes,id
count,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.000000,83123.00000,83123.000000,82867.000000,0.0
mean,52003.849657,39.360249,1191.164455,2.729545,3.060296,2.757612,2.978405,3.204721,3.252060,3.440588,3.359660,3.382060,3.351299,3.632641,3.302516,3.642686,3.28690,14.777763,15.132477,NaN
std,29972.640033,15.102492,997.171944,1.327213,1.523947,1.398630,1.277390,1.330112,1.349006,1.319980,1.332698,1.287719,1.315613,1.180139,1.262015,1.175697,1.31147,37.904165,38.396170,NaN
min,0.000000,7.000000,31.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.00000,0.000000,0.000000,NaN
25%,26120.500000,27.000000,414.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,2.000000,3.000000,3.000000,3.000000,2.00000,0.000000,0.000000,NaN
50%,52004.000000,40.000000,846.000000,3.000000,3.000000,3.000000,3.000000,3.000000,3.000000,4.000000,4.000000,4.000000,4.000000,4.000000,3.000000,4.000000,3.00000,0.000000,0.000000,NaN
75%,77935.500000,51.000000,1744.000000,4.000000,4.000000,4.000000,4.000000,4.000000,4.000000,5.000000,4.000000,4.000000,4.000000,5.000000,4.000000,5.000000,4.00000,12.000000,13.000000,NaN
max,103903.000000,85.000000,4983.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.00000,1592.000000,1584.000000,NaN


False


## 결측치 처리

In [ ]:
# 결측치 이전에 컬럼명에 공백이 많기에 공백 제거.
x_train.columns = x_train.columns.str.replace(r'\s+', '_', regex = True).str.strip()
x_test.columns = x_test.columns.str.replace(r'\s+', '_', regex = True).str.strip()

# 수치형과 문자형으로 나누고 전처리를 하기위해 분할
train_cat = x_train.select_dtypes(include = ['object', 'category'])
train_num = x_train.select_dtypes(include = 'number')
test_cat = x_test.select_dtypes(include = ['object', 'category'])
test_num = x_test.select_dtypes(include = 'number')

cat_lst = x_train.select_dtypes(include = ['object', 'category']).columns.tolist()
num_lst = x_train.select_dtypes(include = 'number').columns.tolist()

# 수치형 변수에 결측치가 있었기에 중앙값으로 대체
train_num = train_num[num_lst].apply(lambda x: x.fillna(x.median(), inplace = False))
test_num = test_num[num_lst].apply(lambda x: x.fillna(x.median(), inplace = False))

## Encoding

In [4]:
from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(
    handle_unknown = 'ignore',
    sparse_output = False
)

# OHE 적용
train_encoded = ohe.fit_transform(train_cat)
test_encoded = ohe.transform(test_cat)

# 데이터프레임으로 적용
train_cat = pd.DataFrame(
    train_encoded,
    columns = ohe.get_feature_names_out(cat_lst),
    index = train_cat.index
)

test_cat = pd.DataFrame(
    test_encoded,
    columns = ohe.get_feature_names_out(cat_lst),
    index = test_cat.index
)

## 결과변수는 라벨 인코딩

In [5]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_train_encoded = le.fit_transform(y_train)

y_train = pd.Series(
    y_train_encoded, 
    index=y_train.index, 
    name='흡연여부')


## 수치형과 범주형 데이터를 하나로 병합

In [6]:
X = pd.concat([train_num, train_cat], axis = 1)
test_final = pd.concat([test_num, test_cat], axis = 1)

## 데이터 분할

In [7]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, y_train, test_size = 0.3, random_state = 42)

## 모델 정의

In [8]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 예측 및 성능지표 출력.

In [9]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

val_pred_label = rf.predict(X_val)
val_pred_proba = rf.predict_proba(X_val)[ : , 1]

print('모델 평가 지표')
print(f'Accuracy    : {accuracy_score(y_val, val_pred_label):.4f}')
print(f'Precision   : {precision_score(y_val, val_pred_label):.4f}')
print(f'Recall      : {recall_score(y_val, val_pred_label):.4f}')
print(f'F1-score    : {f1_score(y_val, val_pred_label):.4f}')
print(f'ROC-AUC     : {roc_auc_score(y_val, val_pred_proba):.4f}')

print('Classification report: ')
print(classification_report(y_val, val_pred_label))

print('Confusion matrix: ')
print(confusion_matrix(y_val, val_pred_label))

모델 평가 지표
Accuracy    : 0.9592
Precision   : 0.9695
Recall      : 0.9348
F1-score    : 0.9518
ROC-AUC     : 0.9928
Classification report: 
              precision    recall  f1-score   support

           0       0.95      0.98      0.96     14175
           1       0.97      0.93      0.95     10762

    accuracy                           0.96     24937
   macro avg       0.96      0.96      0.96     24937
weighted avg       0.96      0.96      0.96     24937

Confusion matrix: 
[[13859   316]
 [  702 10060]]


## CSV 파일 생성

In [10]:
test_pred_label = rf.predict(test_final)
test_pred_proba = rf.predict_proba(test_final)[ : , 1]

pd.DataFrame({'pred' : test_pred_label}).to_csv('label_pred', index = True, index_label = 'ID')
pd.DataFrame({'pred' : test_pred_proba}).to_csv('proba_csv', index = True, index_label = 'ID')

---

# Question 7.
- 수질 음용성 여부 예측
- 데이터를 병합하지 않고 진행.

In [15]:
import pandas as pd

x_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/y_train.csv')
x_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/waters/x_test.csv')

y_train = y_train['Potability']

# 결측치 확인
display(x_train.isna().sum())

# 통계량 확인
display(x_train.describe())

'''
- 오직 수치형 변수로만 구성됨.
- ph, Sulfate, Trihalomethanes 세 개의 컬럼 결측치 처리 필요.
'''


ID                   0
ph                 395
Hardness             0
Solids               0
Chloramines          0
Sulfate            617
Conductivity         0
Organic_carbon       0
Trihalomethanes    132
Turbidity            0
dtype: int64

,ID,ph,Hardness,Solids,Chloramines,Sulfate,Conductivity,Organic_carbon,Trihalomethanes,Turbidity
count,2620.000000,2225.000000,2620.000000,2620.000000,2620.000000,2003.000000,2620.000000,2620.000000,2488.000000,2620.000000
mean,1625.172901,7.059707,196.264418,22093.718726,7.128428,333.128452,426.860672,14.255754,66.362313,3.973541
std,943.704162,1.608470,33.367043,8840.992109,1.589347,41.675257,81.005322,3.335463,16.176955,0.776490
min,0.000000,0.000000,47.432000,728.750830,0.352000,129.000000,181.483754,2.200000,0.738000,1.450000
25%,810.750000,6.069364,176.676887,15675.546419,6.133588,307.592458,366.706407,12.053075,55.614198,3.439540
50%,1614.500000,7.025504,197.103467,20972.316194,7.145240,332.239359,422.145751,14.207656,66.363580,3.969391
75%,2443.250000,8.064653,217.024476,27410.099280,8.124300,358.967592,482.843561,16.509051,77.517441,4.514376
max,3275.000000,14.000000,317.338124,61227.196008,13.043806,481.030642,753.342620,28.300000,120.030077,6.357439


'\n- 오직 수치형 변수로만 구성됨.\n- ph, Sulfate, Trihalomethanes 세 개의 컬럼 결측치 처리 필요.\n'

## 결측치 처리

In [17]:
x_train = x_train[x_train.columns.tolist()].apply(lambda x: x.fillna(x.median(), inplace = False))
x_train.isna().sum()

ID                 0
ph                 0
Hardness           0
Solids             0
Chloramines        0
Sulfate            0
Conductivity       0
Organic_carbon     0
Trihalomethanes    0
Turbidity          0
dtype: int64

## 데이터 분할

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x_train, y_train, test_size = 0.3, random_state = 42)

## 모델 학습

In [19]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(random_state = 23)
rf.fit(X_train, y_train)

,n_estimators,100
,criterion,'gini'
,max_depth,None
,min_samples_split,2
,min_samples_leaf,1
,min_weight_fraction_leaf,0.0
,max_features,'sqrt'
,max_leaf_nodes,None
,min_impurity_decrease,0.0
,bootstrap,True
,oob_score,False


## 예측 수행 및 평가치표 출력

In [21]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix

val_pred_label = rf.predict(X_val)
val_pred_proba = rf.predict_proba(X_val)[ : , 1]

print('모델 예측 성능 평가')
print(f'Accuracy    : {accuracy_score(y_val, val_pred_label):.4f}')
print(f'Precision   : {precision_score(y_val, val_pred_label):.4f}')
print(f'Recall      : {recall_score(y_val, val_pred_label):.4f}')
print(f'F1-score    : {f1_score(y_val, val_pred_label):.4f}')
print(f'ROC-AUC     : {roc_auc_score(y_val, val_pred_proba):.4f}')

print("\n Classification Report: ")
print(classification_report(y_val, val_pred_label))

print("\n Confusion Matrix")
print(confusion_matrix(y_val, val_pred_label))

모델 예측 성능 평가
Accuracy    : 0.6590
Precision   : 0.6403
Recall      : 0.2899
F1-score    : 0.3991
ROC-AUC     : 0.6231

 Classification Report: 
              precision    recall  f1-score   support

           0       0.66      0.90      0.76       479
           1       0.64      0.29      0.40       307

    accuracy                           0.66       786
   macro avg       0.65      0.59      0.58       786
weighted avg       0.65      0.66      0.62       786


 Confusion Matrix
[[429  50]
 [218  89]]


## CSV 파일 생성

In [24]:
test_pred_label = rf.predict(x_test)
test_pred_proba = rf.predict_proba(x_test)[ : , 1]

pd.DataFrame({'pred' : test_pred_label}).to_csv('label_csv', index = True, index_label = 'ID')
pd.DataFrame({'pred' : test_pred_proba}).to_csv('proba_csv', index = True, index_label = 'ID')

---

# Question 8.
- 약물 분류 데이터
- 데이터를 병합하지 않고 진행.